In [ ]:
!unzip /content/Data.zip -d /content/Data

In [112]:
#read the three csv files
import pandas as pd
df_taxi = pd.read_csv('/content/Data/TaxiTrips_BronxOrigin2023.csv')
df_weather = pd.read_csv('/content/Data/Bronx_Weather_Data2023.csv')
df_zone = pd.read_csv('/content/Data/taxi_zone_lookup.csv')

#Already Created Data Preparation


In [113]:
import pandas as pd

# Convert the pickup time from text to datetime type
df_taxi['tpep_pickup_datetime'] = pd.to_datetime(df_taxi['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M')

# Define the start and end of the year
start_date = '2023-01-01 00:00:00'
end_date = '2023-12-31 23:00:00'

# Create a DataFrame with all hourly timestamps for the year
date_range = pd.date_range(start=start_date, end=end_date, freq='H')
hourly_df = pd.DataFrame(date_range, columns=['date'])

# Function to get number of rides and pickup locations within each hour
def get_hourly_data(hour):
    start_time = hour
    end_time = hour + pd.Timedelta(hours=1)

    # Filter records within the hour
    records_within_hour = df_taxi[(df_taxi['tpep_pickup_datetime'] >= start_time) &
                                  (df_taxi['tpep_pickup_datetime'] < end_time)]

    # Calculate required statistics
    record_count = records_within_hour.shape[0]
    total_passengers = records_within_hour['passenger_count'].sum()
    #total_trip_distance = records_within_hour['trip_distance'].sum()

    # Get the list of unique pickup locations
    pickup_locations_list = records_within_hour['PULocationID'].unique().tolist()

    return pd.Series([record_count, pickup_locations_list],
                     index=['number of rides', 'pickup_locations'])

# Apply the function to each hourly timestamp
hourly_df[['number of rides', 'pickup_locations']] = hourly_df['date'].apply(get_hourly_data)

In [114]:
# Extract Month, Day, and Hour
hourly_df['Month'] = hourly_df['date'].dt.month
hourly_df['Day'] = hourly_df['date'].dt.day
hourly_df['Hour'] = hourly_df['date'].dt.hour


# Convert 'date' column to datetime
hourly_df['date'] = pd.to_datetime(hourly_df['date'])

# Create 'weekday' column (1 = Monday, 7 = Sunday)
hourly_df['weekday'] = hourly_df['date'].dt.dayofweek + 1

# Create 'is_weekend' column (True for Saturday and Sunday, False otherwise)
hourly_df['is_weekend'] = (hourly_df['date'].dt.dayofweek >= 5).astype(int)

# Drop the original 'date' column if you no longer need it
#hourly_df = hourly_df.drop(columns=['date', 'Unnamed: 0', 'precipitation'])
hourly_df = hourly_df.drop(columns=['date', 'pickup_locations'])

#hourly_df = hourly_df.drop(columns=['Unnamed: 0', 'precipitation'])
hourly_df.to_csv('/content/output_file.csv', index= False)

In [95]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Separate features (X) and target variable (Y)
X = hourly_df.drop(columns=['number of rides'])   # Exclude 'date' and target column

Y = hourly_df['number of rides']

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Display the resulting splits
print('X_train size is {}'.format(X_train.shape))
print('Y_train size is {}'.format(Y_train.shape))

print('X_test size is {}'.format(X_test.shape))
print('Y_test size is {}'.format(Y_test.shape))

X_train size is (7008, 5)
Y_train size is (7008,)
X_test size is (1752, 5)
Y_test size is (1752,)


#Best Model

In [97]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}


# Initialize XGBoost Regressor
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
                           cv=3, scoring='neg_mean_squared_error',
                           verbose=1, n_jobs=-1)

# Fit Grid Search to training data
grid_search.fit(X_train, Y_train)

# Get the best model and parameters
best_xgb_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Parameters:", best_params)


# Make predictions with the best model
Y_pred = best_xgb_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print("Mean Squared Error (Best XGBoost):", mse)
print("R^2 Score (Best XGBoost):", r2)


Fitting 3 folds for each of 243 candidates, totalling 729 fits
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.9}
Mean Squared Error (Best XGBoost): 11.46371158685731
R^2 Score (Best XGBoost): 0.7321576919466817


#Forecasting the First Week of September 2024

For our forecasting model, we do not need to consider the following factors:

1)Weather: Previous analyses, as demonstrated in earlier Jupyter notebooks, have shown that weather features contribute minimally to prediction accuracy. They are independent of other features and do not significantly enhance the model's performance.

2)Location: All locations are within the Bronx. Additionally, during testing, we may not know the specific location for predictions. Therefore, it is unnecessary to include location data in the training phase, as it does not contribute to predicting the number of rides.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Prepare test data for the first week of September
start_date = datetime(year=2023, month=9, day=1)
end_date = start_date + timedelta(days=6)
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

test_data = pd.DataFrame({
    'Month': date_range.month,
    'Day': date_range.day,
    'Hour': date_range.hour,
    'weekday': date_range.weekday + 1,  # Directly use the 'weekday' property
    'is_weekend': (date_range.weekday >= 5).astype(int)  # 1 if weekend, 0 otherwise
})
print(test_data)

In [122]:
# Make predictions using the best XGBoost model
predictions = best_xgb_model.predict(test_data)

# Round the predictions
rounded_predictions = pd.Series(predictions).round()

# Add predictions to the test data DataFrame
test_data['Predicted_number_of_rides'] = rounded_predictions

# Display the predictions
print("Predictions:")
print(test_data)


Predictions:
     Month  Day  Hour  weekday  is_weekend  Predicted_number_of_rides
0        9    1     0        5           0                        4.0
1        9    1     1        5           0                        3.0
2        9    1     2        5           0                        1.0
3        9    1     3        5           0                        1.0
4        9    1     4        5           0                        5.0
..     ...  ...   ...      ...         ...                        ...
140      9    6    20        3           0                        4.0
141      9    6    21        3           0                        5.0
142      9    6    22        3           0                        4.0
143      9    6    23        3           0                        3.0
144      9    7     0        4           0                        2.0

[145 rows x 6 columns]


In [123]:
import pandas as pd

# Load your DataFrame
df_taxi = pd.read_csv('/content/Data/TaxiTrips_BronxOrigin2023.csv')

# Convert pickup and dropoff datetime columns to datetime format
df_taxi['tpep_pickup_datetime'] = pd.to_datetime(df_taxi['tpep_pickup_datetime'])
df_taxi['tpep_dropoff_datetime'] = pd.to_datetime(df_taxi['tpep_dropoff_datetime'])
# Calculate trip duration in minutes
df_taxi['trip_duration'] = (df_taxi['tpep_dropoff_datetime'] - df_taxi['tpep_pickup_datetime']).dt.total_seconds() / 60
# Calculate the average trip duration
average_duration = df_taxi['trip_duration'].mean()

print(f"The average trip duration is {average_duration:.2f} minutes.")


The average trip duration is 30.64 minutes.
